HOMEWORK OPTIMIZATION:

1) Matrix generation & data preparation:

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from keras.datasets import cifar10
import time
from sklearn.datasets import load_digits
from joblib import Parallel, delayed
import tensorflow as tf

d = 1000  # Number of features
K = 50  # Number of classes
m = 1000   # Number of observations

# Sampling A, X, E from a normal distribution N(0,1)
A = np.random.normal(0, 1, (m, d))  # data matrix of 1000x1000
X = np.random.normal(0, 1, (d, K))  # parameters
E = np.random.normal(0, 1, (m, K))  # Noise

# Computing AX + E
AX = A.dot(X) + E

# Checking the shape of the result for verification
print("Shape of AX:", AX.shape)

# Finding the indices of the maximum values along each column (axis 0)
b = np.argmax(AX, axis=1)

# Printing b to verify correct computation
print("b:", b)


Shape of AX: (1000, 50)
b: [37 24 26 45 42 38 24 37 17 43 20 30 46 39 29 43 48 24 26 42 17 39 13 44
 33 21 36 25 42 19 13  0 29 37  7 19  5 17 29  5  4 18 48 18 13 36 13 44
  6  7 16 26 40 20  1 24 39 18  8 19  8  5 39  7 29 38 15 46  3 20 39 41
 19 37 37  8 23 10  0  2 34 49 47 38 23 42 25  7 26  6  7 27 32 13 34  7
 37 45 30 31 32 20 38  8 47  9 44 26 30 34 14 48 27 18 16 11 42 22 23 39
 26 24 13  8 44 45 15 24 32 40 19 20  5 33 35 15 12 18  9 35 37 31  7 32
 10 20 25 25 47 11  6 49 35 17 42 10 41 24 30 32 38 20 27  6  8 23  5 43
  8 48 22 33 13  9 41  6 30 41 38 10 11 47 22 43 21 46 14  8 44 26  8 38
  3 43 35  0 18 14 35  3 41  5  1 17  1 10  9 13 37  1 44 47 19 24 19 17
  5 44 31  7 28  4 48  9 46  5 14 21 48 10 18 11 33 11 18 14 17 42  3 29
  0 25 34 23 11 10  0 15 13 35  2 34 41 43 49 43 43 31 29 40 29 13 13 23
  9 17 29 28  8 36 43 30 12 16  7 27 23  7 47 12  5 34 38 31 25 14 39 11
  1 14 40 39 49 44  9 19 18 33  4 43 10  0 34 18 10 33  4 31 23 37 12  7
 46 40 22 20  9  9 24 18

In [7]:
# Funzione softmax
def softmax(scores):
    exp_scores = np.exp(scores - np.max(scores, axis=0, keepdims=True))
    return exp_scores / np.sum(exp_scores, axis=0, keepdims=True)

In [8]:
# Inizializzazione dei pesi
weights = np.random.normal(0, 1, (m, d))

# Tasso di apprendimento e numero di iterazioni
eta = 0.02
num_iterations = 50

# Ciclo di ottimizzazione per la discesa del gradiente
for _ in range(num_iterations):
    scores = weights.dot(X)  # m x K
    probs = softmax(scores)

    # Calcolo del gradiente
    grad = np.zeros_like(weights)
    for k in range(m):
        for i in range(K):
            grad[k] += (probs[k, i] - (b[i] == k)) * X[:, i]
    
    # Aggiornamento dei pesi
    weights -= eta * grad
    print('iter', _, '/', num_iterations)

# Stampa dei pesi finali
print("Final weights:")
print(weights)

iter 0 / 50
iter 1 / 50
iter 2 / 50
iter 3 / 50
iter 4 / 50
iter 5 / 50
iter 6 / 50
iter 7 / 50
iter 8 / 50
iter 9 / 50
iter 10 / 50
iter 11 / 50
iter 12 / 50
iter 13 / 50
iter 14 / 50
iter 15 / 50
iter 16 / 50
iter 17 / 50
iter 18 / 50
iter 19 / 50
iter 20 / 50
iter 21 / 50
iter 22 / 50
iter 23 / 50
iter 24 / 50
iter 25 / 50
iter 26 / 50
iter 27 / 50
iter 28 / 50
iter 29 / 50
iter 30 / 50
iter 31 / 50
iter 32 / 50
iter 33 / 50
iter 34 / 50
iter 35 / 50
iter 36 / 50
iter 37 / 50
iter 38 / 50
iter 39 / 50
iter 40 / 50
iter 41 / 50
iter 42 / 50
iter 43 / 50
iter 44 / 50
iter 45 / 50
iter 46 / 50
iter 47 / 50
iter 48 / 50
iter 49 / 50
Final weights:
[[ 0.36986028  0.28210697  1.02227137 ...  0.93843555  0.28499279
   0.12219891]
 [ 1.9500942   0.57037313 -1.15149732 ...  0.32848853  2.35509575
  -0.41692205]
 [ 0.4112971  -1.00922258 -0.26876957 ... -0.02846488 -0.57209187
   1.04543898]
 ...
 [-0.48513051 -0.13087096 -0.35948492 ...  1.57618863  0.46179894
   0.04404178]
 [-0.21474549  0

In [9]:
weights = np.random.normal(0, 1, (m, d))

# Tasso di apprendimento e numero di iterazioni
eta = 0.02
num_iterations = 50

# Ciclo di ottimizzazione per BCGD con Gauss-Southwell
for _ in range(num_iterations):
    scores = weights.dot(X)  # m x K
    probs = softmax(scores)

    # Calcolo del gradiente completo
    grad = np.zeros_like(weights)
    for k in range(m):
        for i in range(K):
            grad[k] += (probs[k, i] - (b[i] == k)) * X[:, i]

    # Selezionare il blocco con il gradiente massimo in valore assoluto
    block_norms = np.linalg.norm(grad, axis=1)
    block_to_update = np.argmax(block_norms)

    # Aggiornamento solo del blocco selezionato
    weights[block_to_update] -= eta * grad[block_to_update]
    print('iter', _, '/', num_iterations)


# Stampa dei pesi finali
print("Final weights:")
print(weights)


iter 0 / 50
iter 1 / 50
iter 2 / 50
iter 3 / 50
iter 4 / 50
iter 5 / 50
iter 6 / 50
iter 7 / 50
iter 8 / 50
iter 9 / 50
iter 10 / 50
iter 11 / 50
iter 12 / 50
iter 13 / 50
iter 14 / 50
iter 15 / 50
iter 16 / 50
iter 17 / 50
iter 18 / 50
iter 19 / 50
iter 20 / 50
iter 21 / 50
iter 22 / 50
iter 23 / 50
iter 24 / 50
iter 25 / 50
iter 26 / 50
iter 27 / 50
iter 28 / 50
iter 29 / 50
iter 30 / 50
iter 31 / 50
iter 32 / 50
iter 33 / 50
iter 34 / 50
iter 35 / 50
iter 36 / 50
iter 37 / 50
iter 38 / 50
iter 39 / 50
iter 40 / 50
iter 41 / 50
iter 42 / 50
iter 43 / 50
iter 44 / 50
iter 45 / 50
iter 46 / 50
iter 47 / 50
iter 48 / 50
iter 49 / 50
Final weights:
[[ 0.53825694 -1.14213837  1.01710882 ...  0.03763827 -0.51342047
   1.27155045]
 [ 0.27041534 -0.30009229 -1.77385636 ...  1.79158046 -0.36295742
   1.34812424]
 [-0.30815305 -0.86218854 -0.29331221 ...  0.71454063  0.34338403
  -0.23913163]
 ...
 [ 0.50883925  0.82344316 -0.37920583 ...  0.90217782 -0.56782103
   0.72192571]
 [-0.22378951 -1

WINE DATASET multiclass classification task:

In [5]:
# URL for the Wine dataset from the UCI Machine Learning Repository
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"

# Column names for the Wine dataset
columns = [
    "Class",
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue",
    "OD280/OD315 of diluted wines",
    "Proline"
]

# Load the dataset into a pandas DataFrame
data = pd.read_csv(url, header=None, names=columns)

# Display the first few rows of the DataFrame
print(data.head())

   Class  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium  \
0      1    14.23        1.71  2.43               15.6        127   
1      1    13.20        1.78  2.14               11.2        100   
2      1    13.16        2.36  2.67               18.6        101   
3      1    14.37        1.95  2.50               16.8        113   
4      1    13.24        2.59  2.87               21.0        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
0             5.64  1.04                          3.92     1065  
1             4.38  1.05  

In [58]:

# URL per il dataset Covertype dal UCI Machine Learning Repository
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"

# Caricare il dataset in un DataFrame pandas
columns = [
    "Elevation", "Aspect", "Slope",
    "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon",
    "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points"
] + [f"Wilderness_Area_{i}" for i in range(4)] + [f"Soil_Type_{i}" for i in range(40)] + ["Cover_Type"]

data = pd.read_csv(url, header=None, names=columns)

# Visualizzare le prime righe del DataFrame
print(data.head())

# Controllare la presenza di valori mancanti
print(data.isnull().sum())

   Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0       2596      51      3                               258   
1       2590      56      2                               212   
2       2804     139      9                               268   
3       2785     155     18                               242   
4       2595      45      2                               153   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                               0                              510   
1                              -6                              390   
2                              65                             3180   
3                             118                             3090   
4                              -1                              391   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0            221             232            148   
1            220             235            151   
2            234             238   

In [22]:
# Caricamento del dataset CIFAR-10
print("Caricamento del dataset CIFAR-10...")
(start_time, load_time) = (time.time(), 0)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Visualizzare le dimensioni del dataset
print(f'Dimensioni del training set: {X_train.shape}')
print(f'Dimensioni del test set: {X_test.shape}')
load_time = time.time() - start_time
print(f"Dataset caricato in {load_time:.2f} secondi.")

# Preprocessing
print("Preprocessing del dataset...")
(start_time, preprocessing_time) = (time.time(), 0)

# Ridimensionare le immagini in vettori
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

Caricamento del dataset CIFAR-10...
170498071/170498071 [==============================] - 12s 0us/step
Dimensioni del training set: (50000, 32, 32, 3)
Dimensioni del test set: (10000, 32, 32, 3)
Dataset caricato in 14.27 secondi.
Preprocessing del dataset...


In [72]:
print("Caricamento del dataset Digits...")
(start_time, load_time) = (time.time(), 0)
digits = load_digits()
X, y = digits.data, digits.target

# Suddivisione del dataset in training set e test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Visualizzare le dimensioni del dataset
print(f'Dimensioni del training set: {X_train.shape}')
print(f'Dimensioni del test set: {X_test.shape}')
load_time = time.time() - start_time
print(f"Dataset caricato in {load_time:.2f} secondi.")

# Preprocessing
print("Preprocessing del dataset...")
(start_time, preprocessing_time) = (time.time(), 0)

# Normalizzare le immagini
X_train = X_train.astype('float32') / 16
X_test = X_test.astype('float32') / 16

Caricamento del dataset Digits...
Dimensioni del training set: (1437, 64)
Dimensioni del test set: (360, 64)
Dataset caricato in 0.02 secondi.
Preprocessing del dataset...


In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from keras.datasets import cifar10
import time
from sklearn.datasets import load_digits
from joblib import Parallel, delayed
import tensorflow as tf
import scipy.sparse as sp


In [2]:
# Carica il dataset 20 Newsgroups
newsgroups = fetch_20newsgroups(subset='all', categories=None, shuffle=True, random_state=42)

# Trasforma i testi in vettori TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)  # Limitando le caratteristiche per migliorare le prestazioni
X = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target

In [3]:
# Dividi i dati in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Stampa le dimensioni dei dati trasformati
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (11307, 1000)
X_test shape: (7539, 1000)
y_train shape: (11307,)
y_test shape: (7539,)


In [4]:
# One-hot encode le etichette
y_train_one_hot = np.eye(np.max(y_train) + 1)[y_train]
y_test_one_hot = np.eye(np.max(y_test) + 1)[y_test]

In [5]:
# Standardizzare le caratteristiche
scaler = StandardScaler(with_mean=False)  # with_mean=False per dati sparsi
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Dividere il training set in training e validation set
X_val, X_test, y_val_one_hot, y_test_one_hot = train_test_split(X_test, y_test_one_hot, test_size=0.5, random_state=42)

X_train = sp.csr_matrix(X_train)
y_train_one_hot = sp.csr_matrix(y_train_one_hot)
X_val = sp.csr_matrix(X_val)
y_val_one_hot = sp.csr_matrix(y_val_one_hot)
X_test = sp.csr_matrix(X_test)
y_test_one_hot = sp.csr_matrix(y_test_one_hot)

# Stampa le dimensioni dei dati trasformati
print(f'X_train shape: {X_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')
print(f'y_val_one_hot shape: {y_val_one_hot.shape}')
print(f'y_test_one_hot shape: {y_test_one_hot.shape}')

X_train shape: (11307, 1000)
X_val shape: (3769, 1000)
X_test shape: (3770, 1000)
y_train_one_hot shape: (11307, 20)
y_val_one_hot shape: (3769, 20)
y_test_one_hot shape: (3770, 20)


In [15]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def compute_cost(probs, y_sparse):
    m = y_sparse.shape[0]
    log_likelihood = -np.log(probs[np.arange(m), y_sparse.indices])
    loss = np.sum(log_likelihood) / m
    return loss

def compute_accuracy(X, y_sparse, weights):
    y_pred = predict(X, weights)
    y_true = y_sparse.indices[y_sparse.indptr[:-1]]
    accuracy = np.mean(y_pred == y_true)
    return accuracy

def gradient_descent(X, y, X_val, y_val, weights, learning_rate, num_epochs, batch_size):
    m, n = X.shape
    for epoch in range(num_epochs):
        indices = np.arange(m)
        np.random.shuffle(indices)
        for i in range(0, m, batch_size):
            batch_indices = indices[i:i + batch_size]
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            
            logits = X_batch.dot(weights)
            probs = softmax(logits)
            gradient = X_batch.T.dot(probs - y_batch) / batch_indices.shape[0]
            weights -= learning_rate * gradient
        
        if epoch % 5 == 0:
            train_cost = compute_cost(softmax(X.dot(weights)), y)
            val_cost = compute_cost(softmax(X_val.dot(weights)), y_val)
            train_accuracy = compute_accuracy(X, y, weights)
            val_accuracy = compute_accuracy(X_val, y_val, weights)
            print(f'Epoch {epoch} completed')
            print(f'Train Cost: {train_cost}, Train Accuracy: {train_accuracy}')
            print(f'Validation Cost: {val_cost}, Validation Accuracy: {val_accuracy}')
    
    return weights

def block_coordinate_gradient_descent(X, y, X_val, y_val, weights, learning_rate, num_epochs, batch_size, block_size):
    m, n = X.shape
    n_blocks = n // block_size

    for epoch in range(num_epochs):
        indices = np.arange(m)
        np.random.shuffle(indices)

        for i in range(0, m, batch_size):
            batch_indices = indices[i:i + batch_size]
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]

            for block in range(n_blocks):
                start = block * block_size
                end = (block + 1) * block_size

                X_block = X_batch[:, start:end]
                W_block = weights[start:end]

                logits = X_batch.dot(weights)
                probs = softmax(logits)
                
                gradient = X_block.T.dot(probs - y_batch) / batch_indices.shape[0]
                W_block -= learning_rate * gradient

                weights[start:end] = W_block
        
        if epoch % 5 == 0:
            train_cost = compute_cost(softmax(X.dot(weights)), y)
            val_cost = compute_cost(softmax(X_val.dot(weights)), y_val)
            train_accuracy = compute_accuracy(X, y, weights)
            val_accuracy = compute_accuracy(X_val, y_val, weights)
            print(f'Epoch {epoch} completed')
            print(f'Train Cost: {train_cost}, Train Accuracy: {train_accuracy}')
            print(f'Validation Cost: {val_cost}, Validation Accuracy: {val_accuracy}')


    return weights

def predict(X, weights):
    logits = X.dot(weights)
    return np.argmax(logits, axis=1)

In [24]:
n_features = X_train.shape[1]
n_classes = y_train_one_hot.shape[1]
learning_rate = 0.005
num_epochs = 50
batch_size = 64
block_size = 64

In [25]:
weights = np.zeros((n_features, n_classes), dtype=np.float32)
weights = gradient_descent(X_train, y_train_one_hot, X_val, y_val_one_hot, weights, learning_rate, num_epochs, batch_size)

Epoch 0 completed
Train Cost: 1.9847935393026381, Train Accuracy: 0.7452020872026178
Validation Cost: 2.0323528341533015, Validation Accuracy: 0.703369594056779
Epoch 2 completed
Train Cost: 1.237264455865634, Train Accuracy: 0.7866808171928894
Validation Cost: 1.3374788852851505, Validation Accuracy: 0.7442292385248076
Epoch 4 completed
Train Cost: 0.9772284541207873, Train Accuracy: 0.8097638630936588
Validation Cost: 1.1131920029469862, Validation Accuracy: 0.7529848766250995
Epoch 6 completed
Train Cost: 0.8404420108978072, Train Accuracy: 0.8267444945608915
Validation Cost: 1.0042369620415548, Validation Accuracy: 0.7625364818254179
Epoch 8 completed
Train Cost: 0.7522400441555318, Train Accuracy: 0.8399221721057751
Validation Cost: 0.9406807266840748, Validation Accuracy: 0.7665163173255506
Epoch 10 completed
Train Cost: 0.6886981079162874, Train Accuracy: 0.8512425930839304
Validation Cost: 0.8984517395599237, Validation Accuracy: 0.7704961528256832
Epoch 12 completed
Train Cost

In [26]:
weights_bcgd = np.zeros((n_features, n_classes), dtype=np.float32)
weights_bcgd = block_coordinate_gradient_descent(X_train, y_train_one_hot, X_val, y_val_one_hot, weights, learning_rate, num_epochs, batch_size, block_size)

Epoch 0 completed
Train Cost: 0.3361444309603628, Train Accuracy: 0.9334925267533386
Validation Cost: 0.7780619729046099, Validation Accuracy: 0.7673122844255771
Epoch 2 completed
Train Cost: 0.32984901973164926, Train Accuracy: 0.934907579375608
Validation Cost: 0.7792328782922795, Validation Accuracy: 0.766781639692226
Epoch 4 completed
Train Cost: 0.32385670888294177, Train Accuracy: 0.9368532767312284
Validation Cost: 0.7803446249817895, Validation Accuracy: 0.765720350225524
Epoch 6 completed
Train Cost: 0.31814342684588204, Train Accuracy: 0.938179888564606
Validation Cost: 0.7816772757326858, Validation Accuracy: 0.765189705492173
Epoch 8 completed
Train Cost: 0.31268303437777406, Train Accuracy: 0.9400371451313345
Validation Cost: 0.782748562362166, Validation Accuracy: 0.7654550278588484
Epoch 10 completed
Train Cost: 0.3074688495778386, Train Accuracy: 0.9412753161758203
Validation Cost: 0.7841076544986113, Validation Accuracy: 0.7665163173255506
Epoch 12 completed
Train Cost

In [27]:
y_test = np.argmax(y_test_one_hot, axis=1)

y_pred = predict(X_test, weights)
if len(y_pred) != y_test_one_hot.shape[0]:
    raise ValueError(f'Shape mismatch: y_pred has shape {y_pred.shape}, y_test has shape {y_test_one_hot.shape[0]}')
accuracy = np.mean(y_pred == y_test)
print(f'Accuracy with classic gradient descent: {accuracy}')

# Predict using block coordinate gradient descent weights
y_pred_bcgd = predict(X_test, weights_bcgd)
if len(y_pred_bcgd) != y_test_one_hot.shape[0]:
    raise ValueError(f'Shape mismatch: y_pred_bcgd has shape {y_pred_bcgd.shape}, y_test has shape {y_test_one_hot.shape[0]}')
accuracy_bcgd = np.mean(y_pred_bcgd == y_test)
print(f'Accuracy with block coordinate gradient descent: {accuracy_bcgd}')

Accuracy with classic gradient descent: 0.05075670693524897
Accuracy with block coordinate gradient descent: 0.05075670693524897
